In [1]:
import module_loader
import pandas as pd
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

In [3]:
def exp(x):
    if isinstance(x, Dual):
        return x.__exp__()
    return math.exp(x)


def log(x):
    if isinstance(x, Dual):
        return x.__log__()
    return math.log(x)


def interpolate(x, x_1, y_1, x_2, y_2, interpolation, start=None):
    if interpolation == "linear":
        op = lambda z: z
    elif interpolation == "log_linear":
        op, y_1, y_2 = exp, log(y_1), log(y_2)
    elif interpolation == "linear_zero_rate":
        y_1 = log(y_1) / ((start - x_1) / timedelta(days=365))
        y_2 = log(y_2) / ((start - x_2) / timedelta(days=365))
        op = lambda z: exp((start-x)/timedelta(days=365) * z)
    ret = op(y_1 + (y_2 - y_1) * (x - x_1) / (x_2 - x_1))
    return ret

In [2]:
class Curve:

    def __init__(self, nodes: dict, interpolation: str, **kwargs):
        self.nodes = deepcopy(nodes)
        self.interpolation = interpolation

    def __getitem__(self, date: datetime):
        node_dates = list(self.nodes.keys())
        for i, node_date_1 in enumerate(node_dates[1:]):
            if date <= node_date_1 or i == len(node_dates) - 2:
                node_date_0 = node_dates[i]
                return interpolate(
                    date,
                    node_date_0,
                    self.nodes[node_date_0],
                    node_date_1,
                    self.nodes[node_date_1],
                    self.interpolation,
                    node_dates[0]
                )

    def __repr__(self):
        output = ""
        for k, v in self.nodes.items():
            output += f"{k.strftime('%Y-%b-%d')}: {v:.6f}\n"
        return output


In [ ]:
class Swap(Covar_, PCA_, Margin_):

    def __init__(
        self,
        start: datetime,
        tenor: int,
        period_fix: int,
        period_float: int,
        days: bool = False,
        fixed_rate: float = None,
        notional: float = None,
    ):
        self.add_op = add_days if days else add_months
        self.start = start
        self.end = self.add_op(start, tenor)
        self.schedule_fix = Schedule(start, tenor, period_fix, days=days)
        self.schedule_float = Schedule(start, tenor, period_float, days=days)
        self.fixed_rate = fixed_rate
        self.notional = 1e6 if notional is None else notional

    def __repr__(self):
        return f"<Swap: {self.start.strftime('%Y-%m-%d')} -> " \
               f"{self.end.strftime('%Y-%m-%d')}>"

    def analytic_delta(self, curve: Curve, leg: str = "fix"):
        delta = 0
        for period in getattr(self, f"schedule_{leg}").data:
            delta += curve[period[1]] * period[2]
        return delta * self.notional / 10000

    def rate(self, curve: Curve):
        if self.notional == 0:
            self.notional = 1
            analytic_delta = self.analytic_delta(curve) * 10000 / self.notional
            rate = (curve[self.start] - curve[self.end]) / analytic_delta
            self.notional = 0
        else:
            analytic_delta = self.analytic_delta(curve) * 10000 / self.notional
            rate = (curve[self.start] - curve[self.end]) / analytic_delta
        return rate * 100

    def npv(self, curve: Curve):
        self.set_fixed_rate(fixed_rate=self.fixed_rate, curve=curve)
        npv = (self.rate(curve) - self.fixed_rate) * self.analytic_delta(curve)
        return npv * 100

    def risk(self, curve: SolvedCurve):
        grad_v_P = np.array([
            [self.npv(curve).dual.get(f"v{i+1}", 0)
             for i in range(curve.n)]
        ]).transpose()
        grad_s_P = np.matmul(curve.grad_s_v, grad_v_P)
        return grad_s_P / 100

    def set_fixed_rate(self, fixed_rate: float = None, *args, **kwargs):
        if fixed_rate is None:
            fixed_rate = self.rate(*args, **kwargs)
            if isinstance(fixed_rate, Dual):
                fixed_rate = fixed_rate.real
        self.fixed_rate = fixed_rate

In [4]:
curve = Curve(interpolation='log_linear',nodes={datetime(2022, 1, 1): 1.00,
    datetime(2023, 1, 1): 0.9975,
    datetime(2024, 1, 1): 0.9945,
    datetime(2025, 1, 1): 0.991})

In [ ]:
curve_dual2 = Curve(interpolation='log_linear',nodes={datetime(2022, 1, 1): Dual(1,{'v0':1}),
    datetime(2023, 1, 1): : Dual(0.9975,{'v1':1}),
    datetime(2024, 1, 1): : Dual(0.9945,{'v2':1})})

In [11]:
curve[datetime(2022,3,15)]

0.9994994992486761

In [6]:
print(curve)

2022-Jan-01: 1.000000
2023-Jan-01: 0.997500
2024-Jan-01: 0.994500
2025-Jan-01: 0.991000



In [8]:
print(repr(curve))

2022-Jan-01: 1.000000
2023-Jan-01: 0.997500
2024-Jan-01: 0.994500
2025-Jan-01: 0.991000



In [9]:
repr(curve)

'2022-Jan-01: 1.000000\n2023-Jan-01: 0.997500\n2024-Jan-01: 0.994500\n2025-Jan-01: 0.991000\n'

In [17]:
swap=Swap(datetime(2022,2,14),4,12,1,notional=1e9)

In [18]:
print(swap,swap.schedule_fix,swap.schedule_float)

<Swap: 2022-02-14 -> 2022-06-14> period start | period end | period DCF
2022-Feb-14 | 2022-Jun-14 | 0.328767
 period start | period end | period DCF
2022-Feb-14 | 2022-Mar-14 | 0.076712
2022-Mar-14 | 2022-Apr-14 | 0.084932
2022-Apr-14 | 2022-May-14 | 0.082192
2022-May-14 | 2022-Jun-14 | 0.084932



In [19]:
swap.analytic_delta(curve)

32839.756869237724

[[datetime.datetime(2022, 2, 14, 0, 0),
  datetime.datetime(2022, 6, 14, 0, 0),
  0.3287671232876712]]

In [22]:
delta = 0
for period in swap.schedule_fix.data:
    delta += curve[period[1]] * period[2]

In [23]:
delta

0.3283975686923773

In [24]:
swap.schedule_fix.data

[[datetime.datetime(2022, 2, 14, 0, 0),
  datetime.datetime(2022, 6, 14, 0, 0),
  0.3287671232876712]]

In [25]:
print(swap.rate(curve))

0.250416047241978
